## Modelling

In this Notebook we can start modelling, with some data from our DB.

- To do this we can connect with our local DB using the `duckdb` library
- When a connection has been made we can start retrieving data from our DB.


### Setup


In [1]:
import duckdb
import polars as pl
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.xgboost

In [2]:
# Use SQL magic
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = True
%config SqlMagic.displaycon = False

%load_ext sql
conn = duckdb.connect(database="../dsp-dagster/data_systems_project.duckdb")
%sql conn --alias duckdb
%sql SHOW ALL TABLES; # shows all available tables

Deploy AI and data apps for free on Ploomber Cloud! Learn more: https://docs.cloud.ploomber.io/en/latest/quickstart/signup.html


Tip: You may define configurations in /home/workspaces/DSP-compose/pyproject.toml or /root/.jupysql/config.

,database,schema,name,column_names,column_types,temporary
0,data_systems_project,joined,incident_deployments_vehicles_weather,"[Station_code, Date, Hour, Dd, Fh, Ff, Fx, T, ...","[BIGINT, DATE, TINYINT, BIGINT, BIGINT, BIGINT...",False
1,data_systems_project,joined,incident_deployments_vehicles_wijken,"[Incident_ID, Date, Incident_Starttime, Incide...","[BIGINT, TIMESTAMP_MS, TIME, TIME, TIME, BIGIN...",False
2,data_systems_project,public,bag_panden,"[geometry, identificatie, rdf_seealso, bouwjaa...","[VARCHAR, VARCHAR, VARCHAR, BIGINT, VARCHAR, V...",False
3,data_systems_project,public,cbs_buurten,"[geometry, buurtcode, buurtnaam, wijkcode, gem...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
4,data_systems_project,public,cbs_wijken,"[geometry, wijkcode, wijknaam, gemeentecode, g...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
5,data_systems_project,public,fire_stations_and_vehicles,"[Fire_Station, Vehicle, Vehicle_Type]","[VARCHAR, VARCHAR, VARCHAR]",False
6,data_systems_project,public,knmi_weather_data,"[station_code, date, hour, DD, FH, FF, FX, T, ...","[BIGINT, VARCHAR, BIGINT, BIGINT, BIGINT, BIGI...",False
7,data_systems_project,public,service_areas,"[H_Verzorgingsgebied_ID, Verzorgingsgebied, LA...","[BIGINT, VARCHAR, DOUBLE, DOUBLE, VARCHAR]",False
8,data_systems_project,public,storm_deployments,"[Deployment_ID, Incident_ID, Vehicle_Type, Veh...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, VA...",False
9,data_systems_project,public,storm_incidents,"[Incident_ID, Date, Incident_Starttime, Incide...","[BIGINT, DATE, TIME, TIME, TIME, BIGINT, VARCH...",False


In [3]:
## We can use SQL magic to retrieve data from our DB like so:
# %sql res << SELECT * FROM joined.deployment_incident_vehicles_weather
# res

In [4]:
# Or the more Pythonic way:

# Here we retrieve a table where KNMI weather data and Fire Department data is combined
df = conn.execute(
    """
    SELECT * FROM joined.incident_deployments_vehicles_weather """
).pl()

# Close the database connection
conn.close()

In [5]:
df.head()

Station_code,Date,Hour,Dd,Fh,Ff,Fx,T,T10n,Td,Sq,Q,Dr,Rh,P,Vv,N,U,Ww,Ix,M,R,S,O,Y,Incident_ID,Incident_Starttime,Incident_Endtime,Incident_Duration,Incident_Priority,Service_Area,Municipality,Damage_Type,LON,LAT,Incident_Endtime_Hour,Incident_Duration_Hour,Incident_Starttime_Minute,Incident_Endtime_Minute,Incident_Duration_Minute,Deployment_ID,Vehicle_Type,Vehicle_Role,Fire_Station,Fire_Station_Service_Status,Driving_Time_To_Incident,Vehicle
i64,date,i8,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,time,time,time,i64,str,str,str,f64,f64,i8,i8,i8,i8,i8,i64,str,str,str,str,str,str
240,2005-01-01,1,260,40,30,60,68,null,57,0,0,0,0,10246,57,8,93,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,2,230,30,30,60,65,null,52,0,0,0,0,10244,58,8,91,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,3,230,40,30,50,43,null,34,0,0,0,0,10241,40,1,94,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,4,220,40,40,50,38,null,32,0,0,0,0,10239,12,0,96,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
240,2005-01-01,5,230,40,40,50,38,null,34,0,0,0,0,10237,14,3,97,10,7,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
def plot_feature_importances(
    model, feature_names, top_n=20, title="Feature Importances"
):
    """
    Plots the top n feature importances in a horizontal bar chart.

    :param model: The trained model
    :param feature_names: List of feature names
    :param top_n: Number of top features to display
    :param title: Title of the plot
    """
    # Extract feature importances
    importances = model.feature_importances_

    # Create a DataFrame and sort it based on importances
    feature_importance_df = pd.DataFrame(
        {"Feature": feature_names, "Importance": importances}
    )
    feature_importance_df = feature_importance_df.sort_values(
        by="Importance", ascending=False
    ).head(top_n)

    # Plotting
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=feature_importance_df, y="Feature", x="Importance", palette="viridis"
    )
    plt.title(title)
    plt.xlabel("Relative Importance")
    plt.ylabel("Feature")
    plt.tight_layout()

    return plt

### XGBoost


In [ ]:
# Select only the relevant columns
weather_cols = [
    "Dd",
    "Fh",
    "Ff",
    "Fx",
    "T",
    "T10n",
    "Td",
    "Sq",
    "Q",
    "Dr",
    "Rh",
    "P",
    "Vv",
    "N",
    "U",
    "Ww",
    "Ix",
    "M",
    "R",
    "S",
    "O",
    "Y",
]
group_cols = ["Date", "Hour", "Service_Area", "Damage_Type"] + weather_cols

# Aggregate data
agg_df = (
    df.groupby(group_cols)
    .agg(Incident_Count=pl.count("Incident_ID"))
    .sort(["Date", "Hour"])
)


# Drop Date and Hour columns if not needed
agg_df = agg_df.drop(["Date", "Hour"])

# Encode categorical variables using one-hot encoding
agg_df = agg_df.to_dummies(columns=["Service_Area", "Damage_Type"])

# Splitting the features and target variable
y = agg_df["Incident_Count"]
X = agg_df.drop("Incident_Count")

# Convert to Pandas DataFrame for compatibility with scikit-learn
X_pd = X.to_pandas()
y_pd = y.to_pandas()

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_pd, y_pd, test_size=0.2, random_state=42
)

# Train XGBoost model
model = xgb.XGBRegressor(
    objective="count:poisson"
)  # Using Poisson regression for count data

model.fit(X_train, y_train)

# Make predictions and calculate metrics
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

# Plot feature importances
feature_importances = model.feature_importances_


print(feature_importances)
plot_feature_importances(model, X_train.columns, top_n=20)


# Set the MLflow tracking URI
mlflow.set_tracking_uri("http://dsp-mlflow:5001")

# Start an MLFlow run
with mlflow.start_run(run_name="Incident Prediction Model"):
    # Log model
    mlflow.xgboost.log_model(model, "xgboost-model")

    # Log parameters
    mlflow.log_params(model.get_params())

    # Log metrics
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)